In [51]:
import numpy as np
import pandas as pd
import math
import torch
import os

# Load datasets

In [125]:
##########################################################################################################################################
################################################################## MNIST #################################################################
##########################################################################################################################################

os.chdir('c:\\Users\\Guillaume\\ViT_vs_Performers\\Data')
data = np.load('mnist.npz')
print(data.files)
print(data['x_train'][0])

x_train = data['x_train']
y_train = data['y_train']
x_test = data['x_test']
y_test = data['y_test']

## Changing to pytorch tensors

x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train)
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test)

## Adding the channels dimension

x_train = x_train.unsqueeze(1)
""" y_train = y_train.unsqueeze(1) """
x_test = x_test.unsqueeze(1)
""" y_test = y_test.unsqueeze(1) """

##########################################################################################################################################
################################################################## ????? #################################################################
##########################################################################################################################################

['x_test', 'x_train', 'y_train', 'y_test']
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136
  175  26 166 255 247 127   0   0   0   0]
 [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253
  225 172 253 242 195  64   0   0   0   0]
 [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251
   93  82  82  56  39   0   0   0   0   0]
 [  0   0   0   0   0

' y_test = y_test.unsqueeze(1) '

# Running ViT

## Initialization

In [95]:
from __future__ import print_function

import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from linformer import Linformer
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

from vit_pytorch.efficient import ViT

import torch
from tqdm import tqdm

In [128]:
# Training settings
batch_size = 64
epochs = 5
lr = 3e-5
gamma = 0.7
seed = 42

In [60]:
""" device = 'cpu'

train_dir = 'data/train'
test_dir = 'data/test'

train_list = glob.glob(os.path.join(train_dir,'*.jpg'))
test_list = glob.glob(os.path.join(test_dir, '*.jpg')) """

" device = 'cpu'\n\ntrain_dir = 'data/train'\ntest_dir = 'data/test'\n\ntrain_list = glob.glob(os.path.join(train_dir,'*.jpg'))\ntest_list = glob.glob(os.path.join(test_dir, '*.jpg')) "

## First plots

In [34]:
class CatsDogsDataset(Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform

    def __len__(self):
        self.filelength = len(self.file_list)
        return self.filelength

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img)

        label = img_path.split("/")[-1].split(".")[0]
        label = 1 if label == "dog" else 0

        return img_transformed, label

In [36]:
train_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]
)

val_transforms = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ]
)

test_transforms = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ]
)

In [105]:
""" train_data = CatsDogsDataset(train_list, transform=train_transforms)
valid_data = CatsDogsDataset(valid_list, transform=test_transforms)
test_data = CatsDogsDataset(test_list, transform=test_transforms)

train_loader = DataLoader(dataset = train_data, batch_size=batch_size, shuffle=True )
valid_loader = DataLoader(dataset = valid_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_data, batch_size=batch_size, shuffle=True) """

' train_data = CatsDogsDataset(train_list, transform=train_transforms)\nvalid_data = CatsDogsDataset(valid_list, transform=test_transforms)\ntest_data = CatsDogsDataset(test_list, transform=test_transforms)\n\ntrain_loader = DataLoader(dataset = train_data, batch_size=batch_size, shuffle=True )\nvalid_loader = DataLoader(dataset = valid_data, batch_size=batch_size, shuffle=True)\ntest_loader = DataLoader(dataset = test_data, batch_size=batch_size, shuffle=True) '

## Models + Efficient Attention

In [91]:
device = 'cpu'

efficient_transformer = Linformer(
    dim=128,
    seq_len=49+1,  # 7x7 patches + 1 cls-token
    depth=12,
    heads=8,
    k=64
)

### Params decided for MNIST

model = ViT(
    dim=128,
    image_size=28,
    patch_size=4,
    num_classes=10,
    transformer=efficient_transformer,
    channels=1,
).to(device)

In [92]:
### Check sizes for MNIST dataset

print(len(x_train))
print((x_train[0]).size)

60000
784


In [93]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [129]:
# Assuming x_train and y_train are PyTorch tensors, and device is properly set
# Define batch size
batch_size = 64  # You can adjust this according to your memory and needs

# Get the number of batches for training and validation
train_size = x_train.size(0)
test_size = x_test.size(0)
num_train_batches = (train_size + batch_size - 1) // batch_size  # Ceiling division
num_valid_batches = (test_size + batch_size - 1) // batch_size  # Ceiling division

# Loop over epochs
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0

    # Training loop
    model.train()
    for i in tqdm(range(num_train_batches), desc=f"Epoch {epoch+1} Training"):
        start = i * batch_size
        end = min((i + 1) * batch_size, train_size)

        data = x_train[start:end]
        label = y_train[start:end]

        # Forward pass
        output = model(data)
        loss = criterion(output, label)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accuracy
        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / num_train_batches
        epoch_loss += loss / num_train_batches

    # Validation loop
    model.eval()
    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for i in range(num_valid_batches):
            start = i * batch_size
            end = min((i + 1) * batch_size, test_size)

            data = x_test[start:end]
            label = y_test[start:end]

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / num_valid_batches
            epoch_val_loss += val_loss / num_valid_batches

    print(f"Epoch: {epoch+1} - loss: {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss: {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n")


Epoch 1 Training:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 1 Training: 100%|██████████| 938/938 [11:34<00:00,  1.35it/s]


Epoch: 1 - loss: 0.1776 - acc: 0.9479 - val_loss: 0.1772 - val_acc: 0.9461



Epoch 2 Training: 100%|██████████| 938/938 [11:25<00:00,  1.37it/s]


Epoch: 2 - loss: 0.1404 - acc: 0.9589 - val_loss: 0.1441 - val_acc: 0.9547



Epoch 3 Training: 100%|██████████| 938/938 [11:35<00:00,  1.35it/s]


Epoch: 3 - loss: 0.1141 - acc: 0.9664 - val_loss: 0.1358 - val_acc: 0.9568



Epoch 4 Training:  87%|████████▋ | 812/938 [09:51<01:45,  1.19it/s]

In [115]:
data.dtype

torch.uint8

In [116]:
{next(model.parameters()).dtype}

{torch.float32}

In [64]:
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(x_train):
        data = data.to(device)
        label = label.to(device)
        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(x_train)
        epoch_loss += loss / len(x_train)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in x_test:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(x_test)
            epoch_val_loss += val_loss / len(x_test)

    print(f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n")

NameError: name 'train_loader' is not defined

In [ ]:
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(train_loader):
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(train_loader)
        epoch_loss += loss / len(train_loader)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in valid_loader:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(valid_loader)
            epoch_val_loss += val_loss / len(valid_loader)

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )

NameError: name 'train_loader' is not defined